In [3]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks/MinahilAssignment/ShapeDetection

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/MinahilAssignment/Main


**Image Preprocessing**
---

In [31]:
from keras.preprocessing.image import ImageDataGenerator

trainDataGen = ImageDataGenerator(
    rescale=1. / 255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)

ValidationDataGen = ImageDataGenerator(rescale=1. / 255)

In [32]:
trainingSet = trainDataGen.flow_from_directory(
    'shapes/train', target_size=(28, 28), batch_size=1, class_mode='categorical')

ValidationSet = ValidationDataGen.flow_from_directory(
    'shapes/validation', target_size=(28, 28), batch_size=1, class_mode='categorical')

Found 200 images belonging to 2 classes.
Found 120 images belonging to 2 classes.


In [33]:
labels = (trainingSet.class_indices)
print(labels)

{'squares': 0, 'triangles': 1}


**Training the Model**
---

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Activation

In [ ]:
# Initialising the CNN
classifier = Sequential()

classifier.add(Conv2D(filters=32,kernel_size=(3,3),padding='Same', input_shape=(28, 28, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))

classifier.add(Conv2D(filters=64,kernel_size=(2,2),padding='Same', activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2),strides=(2,2)))

classifier.add(Flatten())
classifier.add(Dense(56))
classifier.add(Activation('relu'))
classifier.add(Dense(2, activation='softmax'))

# Compiling the CNN
classifier.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
stepsPerEpoch = len(trainingSet.filenames)
validationSteps = len(ValidationSet.filenames)

modelInfo = classifier.fit(trainingSet, steps_per_epoch=stepsPerEpoch, epochs=25, validation_data=ValidationSet, validation_steps=validationSteps)

classifier.save("ShapeDetection.h5")

Epoch 1/25
200/200 [==============================] - 3s 12ms/step - loss: 0.7218 - accuracy: 0.5250 - val_loss: 0.7128 - val_accuracy: 0.5000
Epoch 2/25
200/200 [==============================] - 2s 12ms/step - loss: 0.7007 - accuracy: 0.4400 - val_loss: 0.6930 - val_accuracy: 0.5000
Epoch 3/25
200/200 [==============================] - 2s 11ms/step - loss: 0.6935 - accuracy: 0.5050 - val_loss: 0.6908 - val_accuracy: 0.5000
Epoch 4/25
200/200 [==============================] - 2s 10ms/step - loss: 0.6937 - accuracy: 0.5300 - val_loss: 0.6723 - val_accuracy: 0.6500
Epoch 5/25
200/200 [==============================] - 2s 11ms/step - loss: 0.6786 - accuracy: 0.6650 - val_loss: 0.5900 - val_accuracy: 0.8000
Epoch 6/25
200/200 [==============================] - 3s 17ms/step - loss: 0.5862 - accuracy: 0.7150 - val_loss: 0.4416 - val_accuracy: 0.8250
Epoch 7/25
200/200 [==============================] - 2s 11ms/step - loss: 0.4846 - accuracy: 0.8150 - val_loss: 0.3531 - val_accuracy: 0.8583

**Loading the trained model**
---

In [7]:
from keras.models import load_model

model = load_model('ShapeDetection.h5')

**Testing the model**
---

In [28]:
import numpy as np

def convertToClass(y_pred):
    if len(y_pred.shape) > 1 and y_pred.shape[1] > 1:
        return np.argmax(y_pred, axis=1)
    return np.array([1 if p > 0.5 else 0 for p in y_pred])

In [17]:
testDataGen = ImageDataGenerator(rescale=1. / 255)
testGenerator = testDataGen.flow_from_directory("shapes/test",
                                                      target_size=(28, 28),
                                                      color_mode="rgb",
                                                      shuffle=False,
                                                      class_mode='categorical',
                                                      batch_size=1)
testSteps = len(testGenerator.filenames)
predict = model.predict_generator(testGenerator, steps=testSteps)

actualClasses = convertToClass(predict)
expectedClasses = testGenerator.classes

Found 60 images belonging to 2 classes.


<ipython-input-17-e7042ec369b9>:9: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  predict = model.predict_generator(testGenerator, steps=testSteps)


In [22]:
from sklearn.metrics import confusion_matrix, accuracy_score

print(confusion_matrix(expectedClasses, actualClasses))
print(accuracy_score(expectedClasses, actualClasses))

[[29  1]
 [ 0 30]]
0.9833333333333333


**Prediction**
---

In [26]:
def getClassName(index):
    for key in labels:
      if index == labels[key]:
        return key

In [39]:
import cv2

img = cv2.imread("shapes/test/triangles/drawing(92).png")
img = cv2.resize(img, (28, 28))
img = np.reshape(img, [1, 28, 28, 3])
predict_x=model.predict(img) 
classes_x=convertToClass(predict_x)

print(getClassName(classes_x))

1/1 [==============================] - 0s 16ms/step
triangles
